# Componentes de una aplicación RAG

**Sumario**

1. Introducción
<br></br>
1. Document loaders
   1. ¿Por qué utilizarlos?
   2. Tipos
   3. Document loaders personalizados
   4. Ejemplos
<br></br>
1. Text splitters
   1. ¿Por qué utilizarlos?
   2. Tipos
   3. Ejemplos
<br></br>
1. Modelo de embeddings
   1. OpenAI
   2. HuggingFace
<br></br>
1. Bases de datos vectoriales
   1. Ejemplo con Chroma

# 1 - Introducción

<table>
    <tr>
        <td><img src="./images_4_2/rag_arquitectura.png" width="1000"/></td>
    </tr>
</table>

# 2 - Document loaders

Los "document loaders" (cargadores de documentos) son componentes diseñados para facilitar la carga, manipulación y gestión eficiente de documentos.

LangChain proporciona una serie de document loaders integrados para cargar datos de texto desde fuentes comunes, como archivos, bases de datos y APIs. También es posible crear document loaders personalizados para cargar datos de fuentes específicas.

## 2.1 - ¿Por qué utilizarlos?

* **Aislamiento de la fuente de datos:** Nos permiten separar las fuentes de datos de la aplicación NLP.

* **Flexibilidad:** LangChain posee una larga lista de loaders ya implementados para una gran variedad de fuentes de datos.

* **Eficiencia:** Los document loaders pueden optimizarse para cargar datos de texto de forma eficiente.

## 2.2 - Tipos

**Archivos**
* HTML
  * [`UnstructuredHTMLLoader`](https://python.langchain.com/docs/modules/data_connection/document_loaders/html)
  * [`WebBaseLoader`](https://python.langchain.com/docs/integrations/document_loaders/web_base)
* CSV
  * [`CSVLoader`](https://python.langchain.com/docs/integrations/document_loaders/csv)
* JSON
  * [`JSONLoader`](https://python.langchain.com/docs/modules/data_connection/document_loaders/json)
* PDF
  * [`PyPDFLoader`](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf#using-pypdf)
  * [`PDFMinerLoader`](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf#using-pdfminer)
  * [`PyMuPDFLoader`](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf#using-pymupdf)
* etc.

**Bases de datos**
* Google BigQuery
  * [`BigQueryLoader`](https://python.langchain.com/docs/integrations/document_loaders/google_bigquery)
* AWS S3
  * [`S3FileLoader`](https://python.langchain.com/docs/integrations/document_loaders/aws_s3_file)
  * [`S3DirectoryLoader`](https://python.langchain.com/docs/integrations/document_loaders/aws_s3_directory)
* Mongo DB
  * [`MongodbLoader`](https://python.langchain.com/docs/integrations/document_loaders/mongodb)
* Pandas DataFrame
  * [`DataFrameLoader`](https://python.langchain.com/docs/integrations/document_loaders/pandas_dataframe)
* etc.

**APIs**
* Wikipedia
  * [`WikipediaLoader`](https://python.langchain.com/docs/integrations/document_loaders/wikipedia)
* GitHub
  * [`GitHubIssuesLoader`](https://python.langchain.com/docs/integrations/document_loaders/github)
* Arxiv
  * [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv)
* Telegram
  * [`TelegramchatApiLoader`](https://python.langchain.com/docs/integrations/document_loaders/telegram)
* Twitter (X)
  * [`TwitterTweetLoader`](https://python.langchain.com/docs/integrations/document_loaders/twitter)
* etc.

 ## 2.3 - Document loaders personalizados

También es posible crear document loaders personalizados para cargar datos de fuentes específicas. En este caso, tenemos varios opciones:

**Heredar la clase `BaseDocumentLoader`**. Es el método general, para diferentes tipos de documentos.

```python
abstract class BaseDocumentLoader implements DocumentLoader {
  abstract load(): Promise<Document[]>;
}
```

**Heredar la clase `TextLoader`**. Específico para archivos de texto, pero que tengan un formato especifico.

```python
abstract class TextLoader extends BaseDocumentLoader {
  abstract parse(raw: string): Promise<string[]>;
}
```

**Heredar la clase `BufferLoader`**. Específico para archivos binarios. 
La clase `BufferLoader` se encarga de leer el archivo, por lo que todo lo que tenemos que hacer es implementar el método de parseo.

```python
abstract class BufferLoader extends BaseDocumentLoader {
  abstract parse(
    raw: Buffer,
    metadata: Document["metadata"]
  ): Promise<Document[]>;
}
```

## 2.4 - Ejemplos

### 2.4.1 - `PyMuPDFLoader`

`pip install pymupdf`

In [ ]:
from langchain.document_loaders import PyMuPDFLoader

pdf_loader = PyMuPDFLoader("data/SA_microsoft.pdf")
pdf_docs = pdf_loader.load()

Nos devuelve una lista de objetos `Document`, donde cada documento se corresponde con una página del PDF

In [ ]:
len(pdf_docs)

Hay dos claves relevantes en los documentos:
* `page_content`. Contiene el contenido en formao string
* `metadata`. Los metadatos del documento

In [ ]:
# Metadatos:
pdf_docs[0].metadata

In [ ]:
pdf_docs[0].page_content

### 2.4.2 - `WebBaseLoader`

In [ ]:
from langchain.document_loaders import WebBaseLoader

web_loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
web_docs = web_loader.load()

Descarga la pagina web completa en un solo documento

In [ ]:
len(web_docs)

In [ ]:
web_docs[0].metadata

In [ ]:
web_docs[0].page_content

Si bien este loader es capaz de extraer el texto directamente a partir de una página HTML con una sola línea de código, el texto resultante puede requerir limpieza antes de ser utilizado para entrenar o para inferencia con un LLM.

Por ejemplo, en este caso como mínimo deberiamos limpiar el número de saltos de página...

### 2.4.3 - `WikipediaLoader`

`pip install wikipedia`

En este caso, le hacemos una query a Wikipedia (como si usaramos la parte de "búsqueda" de la página web) y recuperamos como máximo los dos primeros documentos:

In [ ]:
from langchain.document_loaders import WikipediaLoader

wiki_docs = WikipediaLoader(query="HUNTER X HUNTER", load_max_docs=2).load()

len(wiki_docs)

En este caso, podemos ver que nos devuelve (a menos que cambiemos el idioma de la query intrínseca) la página del manga, así como la página de los personajes principales en inglés:

In [ ]:
wiki_docs[0].metadata

In [ ]:
wiki_docs[1].metadata

In [ ]:
wiki_docs[0].page_content

# 3 - Text splitter

Los "text splitters" (separadores de texto) son componentes diseñados para dividir los textos cargados en trozos que quepan en la ventana de contexto de un modelo de embeddings o un LLM.

LangChain proporciona varios text splitters para texto general, HTML y código.

## 3.1 - ¿Por qué utilizarlos?

A primer a vista, la idea de dividir el texto en trozos pequeños pueda parecer simple. Sin embargo, hay mucho potencial de complejidad ya que por lo general **queremos mantener juntos los trozos de texto que se encuentren relacionados semánticamente**. Lo que significa "relacionado semánticamente" podría depender del tipo de texto (no es lo mismo el texto de una novela que el de un código Python). 

Distinguimos dos características principales en un "text splitter" (separador de texto):
* Cómo se divide el texto
* Cómo se mide el tamaño del trozo

## 3.2 - Tipos

| Nombre    | Divide en                        | Agrega Metadatos | Descripción                                                                                                                              |
|-----------|------------------------------------|-------------------|------------------------------------------------------------------------------------------------------------------------------------------|
| [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter) | Una lista de caracteres definidos por el usuario |               | Divide el texto de forma recursiva. La división recursiva tiene como objetivo mantener piezas de texto relacionadas entre sí. Esta es la forma recomendada de comenzar a dividir el texto. |
| [`HTMLHeaderTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/HTML_header_metadata)      | Caracteres específicos de HTML       | ✅            | Divide el texto según caracteres específicos de HTML. Notablemente, esto agrega información relevante sobre de dónde provino ese fragmento (basado en HTML).   |
| [`MarkdownHeaderTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/markdown_header_metadata)  | Caracteres específicos de Markdown   | ✅            | Divide el texto según caracteres específicos de Markdown. Notablemente, esto agrega información relevante sobre de dónde provino ese fragmento (basado en Markdown). |
| [Código](https://python.langchain.com/docs/modules/data_connection/document_transformers/code_splitter)    | Caracteres específicos de código (e.g., Python) |               | Divide el texto según caracteres específicos de lenguajes de programación. Hay disponibles 15 lenguajes diferentes para elegir.           |
| [Token](https://python.langchain.com/docs/modules/data_connection/document_transformers/split_by_token)     | Tokens                             |               | Divide el texto en tokens. Existen múltiples formas diferentes de medir los tokens.                                                        |
| [`CharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/character_text_splitter)  | Un carácter definido por el usuario |               | Divide el texto según un carácter definido por el usuario (e.g., `\n`). Uno de los métodos más simples.                                               |


## 3.3 - Ejemplos

### 3.3.1 - `RecursiveCharacterTextSplitter`


Este separador de texto es el más recomendado para texto genérico. Intenta dividir el texto en orden hasta que los trozos sean lo suficientemente pequeños. 

Está parametrizado por una lista de caracteres. La lista predeterminada es [`\n\n`, `\n`, `" "`, `""`]` . Esto tiene el efecto de intentar mantener todos los párrafos (y luego las oraciones, y luego las palabras) juntos tanto como sea posible, ya que esos serían genéricamente los trozos de texto semánticamente más relacionados.

In [ ]:
text = pdf_docs[1].page_content
print(text)

**Nota:** En este caso, cada linea del documento PDF se ha considerado un párrafo, lo que hace que el `RecursiveCharacterTextSplitter` no funcione tan bien como nos gustaria:

<table>
    <tr>
        <td><img src="./images_4_2/problemas_fin_de_parrafo.png" width="600"/></td>
    </tr>
</table>




In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    # Ponemos un chunk size muy pequeño, simplemente para mostrar la funcionalidad
    chunk_size=100, # caracteres máximos
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
chunks = recursive_text_splitter.split_text(text)
for i in range(0, 3):
    print(f"Chunk #{i}\n")
    print(chunks[i])
    print("\n")

### 3.3.1 - Dividir por tokens usando `tiktoken`

`pip install tiktoken`

In [ ]:
from langchain.text_splitter import TokenTextSplitter

tiktoken_text_splitter = TokenTextSplitter(
    chunk_size=100, # número máximo de tokens
    chunk_overlap=0
)

In [ ]:
chunks = tiktoken_text_splitter.split_text(text)
for i in range(0, 2):
    print(f"Chunk #{i}\n")
    print(chunks[i])
    print("\n")

En este caso, funciona mejor porque ignora los saltos de página erroneos que confunden al anterior text splitter, pero podemos ver que corta frases a la mitad...

### 3.3.1 - Dividir código

Langchain ofrece splitters específicos, aunque tampoco son la panacea. Si cambiamos el chunk size, podemos ver diferencias significativas en la manera en la que divide el texto.

Personalmente creo que **si necesitamos algo realmente inteligente la mejor manera de hacerlo seria con un LLM**.

#### Python

In [ ]:
from langchain.text_splitter import (
    Language,
    RecursiveCharacterTextSplitter,
)

PYTHON_CODE = """
def hello_world():
    print("Hello, World!")

# Llama a la funcion
hello_world()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

#### Markdown

In [ ]:
markdown_text = """
# 🦜️🔗 LangChain

⚡ Building applications with LLMs through composability ⚡

## Quick Install

```bash
# Hopefully this code block isn't split
pip install langchain
```

As an open-source project in a rapidly developing field, we are extremely open to contributions.
"""

md_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN, 
    chunk_size=60, # Si incrementamos a 100, divide el texto por el código
    chunk_overlap=0
)
md_docs = md_splitter.create_documents([markdown_text])
md_docs

# 4 - Modelo de embeddings

La clase Embeddings es una clase diseñada para interactuar con modelos de embedding de texto. Hay muchos proveedores de modelos de embedding (OpenAI, Cohere, Hugging Face, etc.) - esta clase está diseñada para proporcionar una interfaz estándar para todos ellos.

La clase base Embeddings en LangChain proporciona dos métodos: 

* `embed_documents`. Método para convertir documentos. Toma como entrada varios documentos.
* `embed_query`. Método para convertir una consulta. Toma como entrada un texto.

La razon para tener dos métodos separados es que algunos modelos de embedding ofrecen funcionalidad distinta cuando el input es un solo texto o varios documentos.

## 4.1 - OpenAI

In [ ]:
import openai
import os
from langchain.embeddings import AzureOpenAIEmbeddings

# Lee la clave de API desde el archivo de configuración
with open('config.txt') as f:
    config = dict(line.strip().split('=') for line in f)

openai.api_type = "azure"
openai.api_base = "https://gpt3tests.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = config.get("OPENAI_API_KEY", "")

# Nombre del despliegue en mi Azure OpenAI Studio is "TextEmbeddingAda002", el modelo es "text-embedding-ada-002"
engine = "TextEmbeddingAda002"
openai_api_version = "2023-12-04" 

openai_embeddings_model = AzureOpenAIEmbeddings(azure_endpoint=openai.api_base, azure_deployment=engine, openai_api_key=config.get("OPENAI_API_KEY", ""), openai_api_version=openai.api_version)

In [ ]:
texto = "Hola Mundo"
embedded_query = openai_embeddings_model.embed_query(texto)

print(len(embedded_query)) # número de dimensiones del vector de embeddings resultante
print(embedded_query[:5])

In [ ]:
texto_1 = "Hola Mundo"
texto_2 = "Hello World"

textos = [texto_1, texto_2]
embedded_docs = openai_embeddings_model.embed_documents(textos)

print(len(embedded_docs)) # Numero de vectores devueltos
print(len(embedded_docs[0])) # dimensionalidad de los vectores
print(embedded_docs[0][:5])

## 4.2 - HuggingFace

### 4.2.1 - API

Podemos correr los modelos de forma remota mediante la API abierta de HuggingFace (aunque para temas serios tendriamos que pagar).

In [ ]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
huggingface_embeddings_model = HuggingFaceInferenceAPIEmbeddings(
    api_key=config.get("HUGGINGFACE_API_KEY", ""),
    model_name="sentence-transformers/all-MiniLM-l6-v2"
)

embedded_query = huggingface_embeddings_model.embed_query(texto)
print(len(embedded_query)) # número de dimensiones del vector de embeddings resultante
print(embedded_query[:5])

### 4.2.2 - Local

Tambienp odemos correr el modelo de embeddings de forma local (deberia salir el mismo resultado).

`pip install sentence_transformers`

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
local_huggingface_embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embedded_query = local_huggingface_embeddings_model.embed_query(texto)
print(len(embedded_query)) # número de dimensiones del vector de embeddings resultante
print(embedded_query[:5])

# 5 - Base de datos vectorial

Hay gran cantidad de bases de datos vectoriales. Aqui vamos a mostrar la funcionalidad con [Chroma](https://www.trychroma.com/), una alternativa open-source que además no necesita de una API ya que funcionan de forma local (perfecto para hacer pruebas).

Para explorar todas las posibilidades que ofrece LangChain, lo mejor es acudir a [la documentación oficial.](https://python.langchain.com/docs/integrations/vectorstores)

`pip install chromadb`

## 5.1 - Ejemplo con Chroma

En el siguiente ejemplo simple, vamos a coger el earnings-call de Microsoft que tenemos en formato PDF, dividirlo en chunks, pasar esos chunks por el modelo de embeddings y almacenarlo en una instancia de base de datos Chroma local.

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Cargamos el documento
pdf_loader = PyMuPDFLoader("data/SA_microsoft.pdf")
pdf_docs = pdf_loader.load()

# Dividimos el documento en chunks
tiktoken_text_splitter = TokenTextSplitter(
    chunk_size=100, # número máximo de tokens
    chunk_overlap=0
)
chunks = tiktoken_text_splitter.split_documents(pdf_docs)
# El numero cambia segun el chunk_size, el minimo es 21, 
# ya que es el numero de documentos iniciales, 
# correspondiente con el numero de paginas en el PDF
print(f"Hemos generado {len(chunks)} chunks de texto") 

# Cargamos y aplicamos el modelo de embeddings
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Guardamos los chunks en una instancia de Chroma que reside en memoria RAM
db = Chroma.from_documents(chunks, embeddings_model)

Una vez hemos generado la Base de datos con los chunks de texto relevantes. Podemos hacer querys sobre ella de dos maneras:
* `db.similarity_search()`. Directamente con el texto en cuestion (LangChain llama automaticamente al modelo y genera el vector de embeddings)
* `db.similarity_search_by_vector()`. Con el vector de embeddings (habiendolo generado manualmnete)

In [ ]:
# Prueba con el texto directamente:
query = "What are the applications of chat-gpt in cars?"
docs = db.similarity_search(query)

print(docs[0].page_content)

In [ ]:
# Prueba donde primero aplicamos el modelo de embeddings 
# y luego hacemos la query sobre la Base de datos
query = "What are the applications of chat-gpt in cars?"
query_embedding_vector = embeddings_model.embed_query(query)
docs = db.similarity_search_by_vector(query_embedding_vector)

print(docs[0].page_content)

# Siguiente

En la siguiente lección, veremos que es un agente, como utilizarlo, y sus principales usos.